Testing pretrained Detectree2 model weights on OFO data
--------

In [1]:
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectron2.engine import DefaultPredictor
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.train import setup_cfg

ERROR 1: PROJ: proj_create_from_database: Open of /ofo-share/repos-amritha/conda/envs/dtree2/share/proj failed


In [4]:
import rasterio
import geopandas as gpd

In [64]:
trained_model = '/ofo-share/repos-amritha/detectree2-code/230103_randresize_full.pth'  # path to saved weights
img_path = '/ofo-share/scratch-amritha/emerald_point_dtree2/ortho.tif'  # orthomosaic path
tiles_path = '/ofo-share/scratch-amritha/emerald_point_dtree2/'  # folder to save the extracted tiles

In [65]:
data = rasterio.open(img_path)

In [66]:
buffer = 30  # buffer size around each tile
tile_width = 100
tile_height = 100
tiles_path = tiles_path +'tiles_'+ str(buffer) +'_'+ str(tile_width) +'_'+ str(tile_height) +'/'
tile_data(data, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

Tiling to 2 total tiles


/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(


Tiling complete


In [68]:
cfg = setup_cfg(update_model = trained_model)
predict_on_data(tiles_path, predictor = DefaultPredictor(cfg))

/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, m

Predicting 1 files


In [69]:
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

Projecting 1 files


In [70]:
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1) # stitch together predicted crowns from folder w geojson files
clean = clean_crowns(crowns, 0.7, confidence = 0) # set a confidence > 0 to filter out less confident crowns, IoU thresh 0.7

Cleaning 87 crowns
0 / 87 crowns cleaned


In [72]:
clean = clean[clean["Confidence_score"] > 0.5]  

In [75]:
clean = clean.set_geometry(clean.simplify(0.6))

In [78]:
clean.to_file(tiles_path + "/crowns_out_iou07_conf05.gpkg")